In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests

In [2]:
%cd D:\School\540\Data\Final

D:\School\540\Data\Final


### Connecting to an API/Pulling in the Data and Cleaning/Formatting

In [3]:
# Read in file of zip codes to use as a search parameter.
dfzip = pd.read_csv('zip_codes.csv')
dfzip['Zip'] = dfzip['Zip'].astype(str).str.zfill(5)
dfzip

,State,Abbr,Capital,Zip
0,Alabama,AL,Montgomery,36101
1,Alaska,AK,Juneau,99801
2,Arizona,AZ,Phoenix,85001
3,Arkansas,AR,Little Rock,72201
4,California,CA,Sacramento,94203
5,Colorado,CO,Denver,80201
6,Connecticut,CT,Hartford,06101
7,Delaware,DE,Dover,19901
8,Florida,FL,Tallahassee,32301
9,Georgia,GA,Atlanta,30301


In [4]:
api = 'fbSYCkgiTeqM19K2JNZOAF-FW2tmrRGRoaLmDwGCJbFw9IWIgtrl4D7zcLtwthS3WQ-QRt-N4j2mUkFh3hgwFjxiRdeqRsa9EtbsJXskgXtDWTEO-U3260ZummdzYHYx'

headers = {'Authorization': 'Bearer {}'.format(api)}
url = 'https://api.yelp.com/v3/businesses/search'

state_dict = {}

for i in dfzip['Zip']:  # Loop through every state as a Yelp search.
    params = {'term': 'fast food', 'location': i, 'limit': 50}  # Search term 'fast food', zip code as location, return 50 results.
    req = requests.get(url, headers=headers, params=params)
    data_dict = req.json()  # Convert to dictionary.
    
    # Initialize empty lists and dataframe.
    names = []
    count = []
    rating = []
    state_df = pd.DataFrame()
    
    for j in data_dict['businesses']:  # For each business in the return, get the name, number of reviews, and average rating.
        names.append(j['name'])
        count.append(j['review_count'])
        rating.append(j['rating'])
    
    # Insert lists into dataframe.
    state_df['Name'] = names
    state_df['Reviews'] = count
    state_df['Rating'] = rating
    
    state = dfzip.loc[dfzip['Zip'] == i, 'State'].iloc[0]  # Get the state name based on the zip code.
    state_dict[str(state)] = state_df  # Insert dataframe into a dictionary with the state as the key.

state_dict

{'Alabama':                                        Name  Reviews  Rating
 0                                  NYC Gyro      133     4.0
 1                            Hamburger King       46     4.5
 2                         Scott Street Deli       95     4.5
 3                               Chick-fil-A       11     2.5
 4                       Mama's Sack Lunches       39     4.5
 5                              Filet & Vine      183     4.5
 6                           Chris' Hot Dogs      153     3.5
 7                              Jimmy John's        9     2.0
 8                               Chick-fil-A       27     3.5
 9   Zaxby's Chicken Fingers & Buffalo Wings       20     3.0
 10                              Burger King        7     2.0
 11                                Five Guys       17     3.5
 12                                  Wendy's        4     1.5
 13                      Chicken Salad Chick       25     4.0
 14                              Cafe Louisa      169     4

In [5]:
# Initialize empty lists and dataframe.
state = []
tot_rev = []
avg_rat = []
df = pd.DataFrame()

for k,v in state_dict.items():  # Loop through dictionary items to get the 
    state.append(k)  # Add state value to list.
    rev_sum = v['Reviews'].sum()  # Get total number of reviews.
    tot_rev.append(rev_sum)  # Add total reviews to list.
    avg = 0
    for _,row in v.iterrows(): # Loop through each row in the dataframe to find the weighted average based on number of reviews.
        avg += row['Rating']  * row['Reviews'] / rev_sum
    avg_rat.append(avg)  # Add average rating to list.

# Add lists as columns to dataframe.
df['State'] = state
df['Total Reviews'] = tot_rev
df['Average Rating'] = avg_rat
df

,State,Total Reviews,Average Rating
0,Alabama,1526,3.722805
1,Alaska,138,3.311594
2,Arizona,15980,4.085419
3,Arkansas,4075,4.044908
4,California,19159,3.708257
5,Colorado,22811,4.023914
6,Connecticut,4851,3.827046
7,Delaware,1742,3.385763
8,Florida,3804,3.908912
9,Georgia,17217,3.874746


In [6]:
df['Average Rating'] = df['Average Rating'].round(2)  # Round average rating to two decimals.
df

,State,Total Reviews,Average Rating
0,Alabama,1526,3.72
1,Alaska,138,3.31
2,Arizona,15980,4.09
3,Arkansas,4075,4.04
4,California,19159,3.71
5,Colorado,22811,4.02
6,Connecticut,4851,3.83
7,Delaware,1742,3.39
8,Florida,3804,3.91
9,Georgia,17217,3.87


In [8]:
df = df.merge(dfzip, how='left', on='State')
df

,State,Total Reviews,Average Rating,Abbr,Capital,Zip
0,Alabama,1526,3.72,AL,Montgomery,36101
1,Alaska,138,3.31,AK,Juneau,99801
2,Arizona,15980,4.09,AZ,Phoenix,85001
3,Arkansas,4075,4.04,AR,Little Rock,72201
4,California,19159,3.71,CA,Sacramento,94203
5,Colorado,22811,4.02,CO,Denver,80201
6,Connecticut,4851,3.83,CT,Hartford,06101
7,Delaware,1742,3.39,DE,Dover,19901
8,Florida,3804,3.91,FL,Tallahassee,32301
9,Georgia,17217,3.87,GA,Atlanta,30301


In [10]:
#df.to_csv('yelp_data.csv', index=False)  # Write Yelp data to csv.